<a href="https://colab.research.google.com/github/mdehghani86/StorytellingData/blob/main/lab2_mab_combined_json_html.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div style="background: linear-gradient(90deg, #17a2b8 0%, #0e5a63 60%, #0a3d44 100%); color: white; padding: 18px 25px; margin-bottom: 20px;">
    <div style="display: flex; justify-content: space-between; align-items: baseline;">
        <h1 style="font-family: 'Helvetica Neue', sans-serif; font-size: 24px; margin: 0; font-weight: 300;">
            Lab 2: Multi-Armed Bandits
        </h1>
        <span style="font-size: 11px; opacity: 0.9;">© Prof. Dehghani</span>
    </div>
    <p style="font-size: 13px; margin-top: 6px; margin-bottom: 0; opacity: 0.9;">
        IE 7295 Reinforcement Learning | Sutton & Barto Chapter 2 | Intermediate Level | 90 minutes
    </p>
</div>

<div style="background: white; padding: 15px 20px; margin-bottom: 12px; border-left: 3px solid #17a2b8;">
    <h3 style="color: #17a2b8; font-size: 14px; margin: 0 0 8px 0; text-transform: uppercase; letter-spacing: 0.5px;">Background</h3>
    <p style="color: #555; line-height: 1.6; margin: 0; font-size: 13px;">
        The multi-armed bandit problem models decision-making under uncertainty. An agent repeatedly chooses among k actions,
        receiving numerical rewards from stationary probability distributions. The challenge is balancing exploration
        (trying different actions to find the best) with exploitation (choosing the current best action).
        This lab reproduces key results from <a href="http://incompleteideas.net/book/the-book-2nd.html" style="color: #17a2b8;">Sutton & Barto (2018)</a>, Chapter 2.
    </p>
</div>

<table style="width: 100%; border-spacing: 12px;">
<tr>
<td style="background: white; padding: 12px 15px; border-top: 3px solid #17a2b8; vertical-align: top; width: 50%;">
    <h4 style="color: #17a2b8; font-size: 13px; margin: 0 0 8px 0; font-weight: 600;">Learning Objectives</h4>
    <ul style="color: #555; line-height: 1.4; margin: 0; padding-left: 18px; font-size: 12px;">
        <li>Implement the 10-armed testbed</li>
        <li>Compare ε-greedy strategies</li>
        <li>Analyze optimistic initial values</li>
        <li>Reproduce Figures 2.1, 2.2, and 2.3</li>
    </ul>
</td>
<td style="background: white; padding: 12px 15px; border-top: 3px solid #00acc1; vertical-align: top; width: 50%;">
    <h4 style="color: #00acc1; font-size: 13px; margin: 0 0 8px 0; font-weight: 600;">Key Concepts</h4>
    <div style="color: #555; font-size: 12px; line-height: 1.6;">
        <div style="padding: 2px 0;"><code style="background: #e0f7fa; padding: 1px 5px; color: #006064;">q*(a)</code> = true action value</div>
        <div style="padding: 2px 0;"><code style="background: #e0f7fa; padding: 1px 5px; color: #006064;">Qt(a)</code> = estimated value at time t</div>
        <div style="padding: 2px 0;"><code style="background: #e0f7fa; padding: 1px 5px; color: #006064;">ε-greedy</code> = exploration strategy</div>
        <div style="padding: 2px 0;"><code style="background: #e0f7fa; padding: 1px 5px; color: #006064;">α</code> = step-size parameter</div>
    </div>
</td>
</tr>
</table>

## Configuration and Setup

In [ ]:
"""
Cell 1: Environment Setup and Configuration
Purpose: Import libraries and set visualization parameters
"""

import numpy as np
import matplotlib.pyplot as plt
from typing import Tuple, List
import warnings
warnings.filterwarnings('ignore')

# Configurable color scheme - modify these to change plot colors
COLORS = {
    'greedy': '#008000',      # Green for ε=0
    'epsilon_01': '#FF0000',  # Red for ε=0.01
    'epsilon_1': '#0000FF',   # Blue for ε=0.1
    'optimistic': '#00BFFF',  # Cyan for optimistic
    'realistic': '#808080',   # Gray for realistic
    'violin': '#7f7f7f'       # Gray for violin plots
}

# Standard parameters from Sutton & Barto
K = 10          # Number of arms
STEPS = 1000    # Time steps per run
RUNS = 2000     # Number of independent runs

# Configure matplotlib
plt.rcParams['figure.dpi'] = 100
plt.rcParams['font.size'] = 10
plt.rcParams['axes.labelsize'] = 11
plt.rcParams['axes.titlesize'] = 12
plt.rcParams['legend.fontsize'] = 10

def pretty_print(title: str, value: any) -> None:
    """Display formatted output"""
    print(f"\n{'='*50}")
    print(f"{title}: {value}")
    print(f"{'='*50}")

## Part 1: The 10-Armed Testbed (Figure 2.1)

The testbed consists of 10 actions with action values $q_*(a), a = 1, ..., 10$, selected from a normal distribution with mean 0 and variance 1.

In [ ]:
"""
Cell 2: Create and Visualize the 10-Armed Testbed
Purpose: Generate Figure 2.1 showing reward distributions
"""

# Set seed for reproducible example
np.random.seed(0)

# Generate true action values q*(a) ~ N(0, 1)
q_true = np.random.randn(K)

# Create Figure 2.1: Reward distributions
fig, ax = plt.subplots(figsize=(10, 6))

# Generate reward distribution samples for visualization
n_samples = 10000
rewards = np.zeros((n_samples, K))
for i in range(K):
    rewards[:, i] = q_true[i] + np.random.randn(n_samples)

# Create violin plots
parts = ax.violinplot(rewards, positions=range(1, K+1), widths=0.7,
                      showmeans=False, showextrema=False)

# Style the violins
for pc in parts['bodies']:
    pc.set_facecolor(COLORS['violin'])
    pc.set_alpha(0.7)
    pc.set_edgecolor('black')
    pc.set_linewidth(0.5)

# Add horizontal lines for true values
for i in range(K):
    ax.hlines(q_true[i], i+0.7, i+1.3, colors='black', linewidth=1.5)
    ax.text(i+1.4, q_true[i], f'$q_*({i+1})$', fontsize=10, va='center')

# Formatting
ax.set_xlabel('Action', fontsize=12)
ax.set_ylabel('Reward\ndistribution', fontsize=12)
ax.set_xticks(range(1, K+1))
ax.set_xlim(0.5, K+0.5)
ax.set_ylim(-3, 3)
ax.axhline(y=0, color='black', linestyle='--', linewidth=0.5, alpha=0.5)
ax.grid(True, alpha=0.3, axis='y')

plt.title('Figure 2.1: The 10-armed testbed', fontsize=12)
plt.tight_layout()
plt.show()

pretty_print("Optimal action", np.argmax(q_true) + 1)

## Part 2: Core Algorithms

We implement the fundamental components:
- **ε-greedy action selection**: Choose randomly with probability ε, otherwise choose greedily
- **Sample-average update**: $Q_{n+1} = Q_n + \frac{1}{n}[R_n - Q_n]$

In [ ]:
"""
Cell 3: Core Algorithm Implementation
Purpose: Define action selection and value update methods
"""

def create_bandit() -> np.ndarray:
    """Create a new 10-armed bandit problem"""
    return np.random.randn(K)

def get_reward(action: int, q_true: np.ndarray) -> float:
    """Get reward for an action: R ~ N(q*(a), 1)"""
    return q_true[action] + np.random.randn()

def epsilon_greedy(Q: np.ndarray, epsilon: float) -> int:
    """
    ε-greedy action selection

    Args:
        Q: Current action value estimates
        epsilon: Probability of random exploration

    Returns:
        Selected action index
    """
    if np.random.random() < epsilon:
        # Explore: choose random action
        return np.random.randint(K)
    else:
        # Exploit: choose greedy action (break ties randomly)
        max_Q = np.max(Q)
        return np.random.choice(np.where(Q == max_Q)[0])

def update_estimates(Q: np.ndarray, N: np.ndarray, action: int,
                    reward: float, alpha: float = None) -> None:
    """
    Update action value estimates

    Args:
        Q: Action value estimates (modified in place)
        N: Action counts
        action: Selected action
        reward: Observed reward
        alpha: Step size (None for sample average)
    """
    N[action] += 1

    if alpha is None:
        # Sample average: Q = Q + (1/n)[R - Q]
        Q[action] += (reward - Q[action]) / N[action]
    else:
        # Constant step size: Q = Q + α[R - Q]
        Q[action] += alpha * (reward - Q[action])

## Part 3: Comparing ε-greedy Methods (Figure 2.2)

We compare three ε-greedy variants: ε = 0 (greedy), ε = 0.01, and ε = 0.1

In [ ]:
"""
Cell 4: Run ε-greedy Experiments
Purpose: Generate data for Figure 2.2
"""

def run_epsilon_greedy_experiment(epsilon: float, runs: int = 2000,
                                 steps: int = 1000) -> Tuple[np.ndarray, np.ndarray]:
    """
    Run ε-greedy bandit experiment

    Returns:
        (average_rewards, optimal_action_percentage)
    """
    all_rewards = np.zeros((runs, steps))
    all_optimal = np.zeros((runs, steps))

    for run in range(runs):
        # Initialize new problem
        q_true = create_bandit()
        optimal_action = np.argmax(q_true)

        # Initialize estimates
        Q = np.zeros(K)
        N = np.zeros(K)

        # Run steps
        for step in range(steps):
            # Select action
            action = epsilon_greedy(Q, epsilon)

            # Get reward
            reward = get_reward(action, q_true)

            # Update estimates
            update_estimates(Q, N, action, reward)

            # Record results
            all_rewards[run, step] = reward
            all_optimal[run, step] = (action == optimal_action)

    # Return averages
    return np.mean(all_rewards, axis=0), np.mean(all_optimal, axis=0) * 100

# Run experiments for different epsilon values
print("Running experiments for Figure 2.2...")
results = {}

for eps, name in [(0, 'greedy'), (0.01, 'epsilon_01'), (0.1, 'epsilon_1')]:
    print(f"  ε = {eps}...")
    avg_reward, pct_optimal = run_epsilon_greedy_experiment(eps, RUNS, STEPS)
    results[eps] = (avg_reward, pct_optimal, name)

print("Complete!")

In [ ]:
"""
Cell 5: Plot Figure 2.2
Purpose: Reproduce Figure 2.2 from Sutton & Barto
"""

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(8, 8))

# Plot average rewards
for eps in [0.1, 0.01, 0]:  # Order for correct layering
    avg_reward, _, name = results[eps]
    ax1.plot(avg_reward, color=COLORS[name], label=f'ε = {eps}', linewidth=1.5)

ax1.set_ylabel('Average\nreward', fontsize=11)
ax1.set_xlim(0, 1000)
ax1.set_ylim(0, 1.5)
ax1.legend(loc='lower right')
ax1.grid(True, alpha=0.3)

# Plot optimal action percentage
for eps in [0.1, 0.01, 0]:  # Order for correct layering
    _, pct_optimal, name = results[eps]
    label = f'ε = {eps} (greedy)' if eps == 0 else f'ε = {eps}'
    ax2.plot(pct_optimal, color=COLORS[name], label=label, linewidth=1.5)

ax2.set_xlabel('Steps', fontsize=11)
ax2.set_ylabel('%\nOptimal\naction', fontsize=11)
ax2.set_xlim(0, 1000)
ax2.set_ylim(0, 100)
ax2.legend(loc='lower right')
ax2.grid(True, alpha=0.3)

fig.suptitle('Figure 2.2: Average performance of ε-greedy action-value methods\non the 10-armed testbed',
             fontsize=12, y=0.995)
plt.tight_layout()
plt.show()

## Part 4: Optimistic Initial Values (Figure 2.3)

Optimistic initialization encourages exploration even with ε = 0. We compare:
- Optimistic greedy: Q₁ = 5, ε = 0
- Realistic ε-greedy: Q₁ = 0, ε = 0.1

In [ ]:
"""
Cell 6: Optimistic Initial Values Experiment
Purpose: Generate data for Figure 2.3
"""

def run_optimistic_experiment(epsilon: float, initial_Q: float, alpha: float = 0.1,
                             runs: int = 2000, steps: int = 1000) -> np.ndarray:
    """
    Run experiment with specified initial values
    Uses constant step size α = 0.1 as per Figure 2.3

    Returns:
        Percentage of optimal actions at each step
    """
    all_optimal = np.zeros((runs, steps))

    for run in range(runs):
        # Initialize new problem
        q_true = create_bandit()
        optimal_action = np.argmax(q_true)

        # Initialize with specified values
        Q = np.ones(K) * initial_Q
        N = np.zeros(K)

        # Run steps
        for step in range(steps):
            # Select action
            action = epsilon_greedy(Q, epsilon)

            # Get reward
            reward = get_reward(action, q_true)

            # Update with constant step size
            update_estimates(Q, N, action, reward, alpha=alpha)

            # Record if optimal
            all_optimal[run, step] = (action == optimal_action)

    return np.mean(all_optimal, axis=0) * 100

# Run experiments
print("Running experiments for Figure 2.3...")
print("  Optimistic greedy (Q₁=5, ε=0)...")
optimistic_greedy = run_optimistic_experiment(epsilon=0, initial_Q=5, alpha=0.1)

print("  Realistic ε-greedy (Q₁=0, ε=0.1)...")
realistic_egreedy = run_optimistic_experiment(epsilon=0.1, initial_Q=0, alpha=0.1)

print("Complete!")

In [ ]:
"""
Cell 7: Plot Figure 2.3
Purpose: Reproduce Figure 2.3 from Sutton & Barto
"""

plt.figure(figsize=(8, 5))

# Plot optimistic greedy
plt.plot(optimistic_greedy, color=COLORS['optimistic'],
         label='Optimistic, greedy\n$Q_1 = 5, ε = 0$', linewidth=1.5)

# Plot realistic ε-greedy
plt.plot(realistic_egreedy, color=COLORS['realistic'],
         label='Realistic, ε-greedy\n$Q_1 = 0, ε = 0.1$', linewidth=1.5)

plt.xlabel('Steps', fontsize=11)
plt.ylabel('%\nOptimal\naction', fontsize=11)
plt.xlim(0, 1000)
plt.ylim(0, 100)
plt.legend(loc='lower right')
plt.grid(True, alpha=0.3)

plt.title('Figure 2.3: The effect of optimistic initial action-value estimates on the 10-armed testbed.\n' +
          'Both methods used a constant step-size parameter, α = 0.1.',
          fontsize=11)
plt.tight_layout()
plt.show()

## Part 5: Binomial Reward Variant

A simplified variant using binary rewards, useful for modeling click-through rates or conversion problems.

In [ ]:
"""
Cell 8: Binomial Bandit Implementation
Purpose: Demonstrate binary reward variant
"""

def create_binomial_bandit() -> np.ndarray:
    """Create bandit with success probabilities"""
    return np.random.beta(2, 2, K)  # Beta(2,2) gives values around 0.5

def get_binomial_reward(action: int, p_true: np.ndarray) -> int:
    """Binary reward with probability p(a)"""
    return 1 if np.random.random() < p_true[action] else 0

def run_binomial_experiment(epsilon: float, runs: int = 1000,
                          steps: int = 1000) -> Tuple[np.ndarray, np.ndarray]:
    """
    Run binomial bandit experiment
    """
    all_rewards = np.zeros((runs, steps))
    all_optimal = np.zeros((runs, steps))

    for run in range(runs):
        # Initialize problem
        p_true = create_binomial_bandit()
        optimal_action = np.argmax(p_true)

        # Initialize estimates
        Q = np.zeros(K)
        N = np.zeros(K)

        for step in range(steps):
            # Select action
            action = epsilon_greedy(Q, epsilon)

            # Get binary reward
            reward = get_binomial_reward(action, p_true)

            # Update estimates
            update_estimates(Q, N, action, reward)

            # Record results
            all_rewards[run, step] = reward
            all_optimal[run, step] = (action == optimal_action)

    return np.mean(all_rewards, axis=0), np.mean(all_optimal, axis=0) * 100

# Run binomial experiments
print("Running binomial bandit experiments...")
binomial_results = {}

for eps in [0, 0.01, 0.1]:
    print(f"  ε = {eps}...")
    avg_reward, pct_optimal = run_binomial_experiment(eps, runs=1000)
    binomial_results[eps] = (avg_reward, pct_optimal)

print("Complete!")

In [ ]:
"""
Cell 9: Visualize Binomial Results
Purpose: Compare binomial bandit performance
"""

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

# Plot rewards
for eps, (avg_reward, _) in binomial_results.items():
    name = 'greedy' if eps == 0 else f'epsilon_{int(eps*100):02d}'
    ax1.plot(avg_reward, color=COLORS[name], label=f'ε = {eps}', linewidth=1.5)

ax1.set_xlabel('Steps')
ax1.set_ylabel('Average Reward')
ax1.set_title('Binomial Bandit: Average Reward')
ax1.legend(loc='lower right')
ax1.grid(True, alpha=0.3)

# Plot optimal actions
for eps, (_, pct_optimal) in binomial_results.items():
    name = 'greedy' if eps == 0 else f'epsilon_{int(eps*100):02d}'
    ax2.plot(pct_optimal, color=COLORS[name], label=f'ε = {eps}', linewidth=1.5)

ax2.set_xlabel('Steps')
ax2.set_ylabel('% Optimal Action')
ax2.set_title('Binomial Bandit: Optimal Action Selection')
ax2.legend(loc='lower right')
ax2.grid(True, alpha=0.3)

plt.suptitle('Binomial Reward Variant (Binary Outcomes)', fontsize=12, y=1.02)
plt.tight_layout()
plt.show()

<div style="background: #f8f9fa; padding: 15px 20px; margin-top: 30px; border-left: 3px solid #17a2b8;">
    <h3 style="color: #17a2b8; font-size: 14px; margin: 0 0 8px 0; text-transform: uppercase; letter-spacing: 0.5px;">Summary</h3>
    <div style="color: #555; line-height: 1.6; font-size: 13px;">
        <p><strong>Key Findings:</strong></p>
        <ul style="margin: 10px 0; padding-left: 20px;">
            <li>ε-greedy with ε = 0.1 achieves good balance between exploration and exploitation</li>
            <li>Pure greedy (ε = 0) often gets stuck on suboptimal actions</li>
            <li>Optimistic initialization drives initial exploration even without ε-greedy</li>
            <li>Binary rewards (binomial variant) show similar patterns with faster convergence</li>
        </ul>
    </div>
</div>

<div style="background: #fff3e0; padding: 15px 20px; margin-top: 20px; border-left: 3px solid #ff9800;">
    <h3 style="color: #ff9800; font-size: 14px; margin: 0 0 8px 0; text-transform: uppercase; letter-spacing: 0.5px;">Questions for Reflection</h3>
    <ol style="color: #555; line-height: 1.8; margin: 8px 0 0 0; padding-left: 20px; font-size: 13px;">
        <li>How would performance change with different numbers of arms (k = 2, 100, 1000)?</li>
        <li>What happens to optimistic initialization as the number of steps increases?</li>
        <li>How could you adapt ε over time for better performance?</li>
        <li>When would you prefer optimistic initialization over ε-greedy exploration?</li>
    </ol>
</div>

<div style="background: linear-gradient(90deg, #17a2b8 0%, #0e5a63 60%, #0a3d44 100%); color: white; padding: 15px 20px; margin-top: 30px; text-align: center;">
    <p style="margin: 0; font-size: 13px;">End of Lab 2: Multi-Armed Bandits</p>
</div>